## Dry Beans Evaluation and Optimization Lab

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime


import boto3
from sagemaker import get_execution_role
import sagemaker

In [3]:
role = get_execution_role()
bucket='ars-mlops-projects'

### Create and train our "optimized" model (Linear Learner)

In [4]:
train_file = 'dry_beans_train_recordIO_protobuf.data'
training_recordIO_protobuf_location = 's3://{}/dry-beans/data/LinearLearner/{}'.format(bucket, train_file)
print('The Pipe mode recordIO protobuf training data: {}'.format(training_recordIO_protobuf_location))

The Pipe mode recordIO protobuf training data: s3://dry-dataset-1521/algorithms_lab/linearlearner_train/dry_beans_train_recordIO_protobuf.data


In [5]:
validation_file = 'dry_beans_validatioin_recordIO_protobuf.data'
validate_recordIO_protobuf_location = 's3://ars-mlops-projects/dry-beans/data/LinearLearner/{}'.format(validation_file)
print('The Pipe mode recordIO protobuf validation data: {}'.format(validate_recordIO_protobuf_location))

The Pipe mode recordIO protobuf validation data: s3://dry-dataset-1521/algorithms_lab/linearlearner_validation/dry_beans_validatioin_recordIO_protobuf.data


In [6]:
from sagemaker import image_uris
container = image_uris.retrieve('linear-learner', boto3.Session().region_name, '1')

In [7]:
# Create a training job name
job_name = 'dry-beans-linear-learner-job-optimized-{}'.format(datetime.now().strftime("%Y%m%d%H%M%S"))
print('Here is the job name {}'.format(job_name))

# Here is where the model-artifact will be stored
output_location = 's3://ars-mlops-projects/dry-beans/data/LinearLearner/OptimizationOutput/'.format(bucket)

Here is the job name dry-beans-linear-learner-job-optimized-20211028064650


In [8]:
%%time
sess = sagemaker.Session()

# Setup the LinearLeaner algorithm from the ECR container
linear = sagemaker.estimator.Estimator(container,
                                       role, 
                                       instance_count=1, 
                                       instance_type='ml.c4.xlarge',
                                       output_path=output_location,
                                       sagemaker_session=sess,
                                       input_mode='Pipe')
# Setup the hyperparameters
linear.set_hyperparameters( feature_dim=16, # number of attributes (minus the Dry Beans attribute)
                            predictor_type='multiclass_classifier',
                            num_classes=7,
                            early_stopping_patience=3,
                            early_stopping_tolerance=0.001,
                            epochs=15,
                            l1=0.0003891683594472962,
                            learning_rate=0.21329477268698604,
                            loss='auto',
                            mini_batch_size=1032,
                            normalize_data='true',
                            normalize_label='auto',
                            num_models='auto',
                            optimizer='auto',
                            unbias_data='auto',
                            unbias_label='auto',
                            use_bias='true',
                            wd=0.00012190629716256174
                          )


# Launch a training job. This method calls the CreateTrainingJob API call
data_channels = {
    'train': training_recordIO_protobuf_location,
    'validation': validate_recordIO_protobuf_location
}
linear.fit(data_channels, job_name=job_name)

2021-10-28 06:49:08 Starting - Starting the training job...
2021-10-28 06:49:31 Starting - Launching requested ML instancesProfilerReport-1635403748: InProgress
...
2021-10-28 06:50:00 Starting - Preparing the instances for training............
2021-10-28 06:52:04 Downloading - Downloading input data
2021-10-28 06:52:04 Training - Downloading the training image...
2021-10-28 06:52:32 Training - Training image download completed. Training in progress.Docker entrypoint called with argument(s): train
Running default environment configuration script
[10/28/2021 06:52:29 INFO 139735411730240] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': '